## Reaction time demo embedded

## Setup

In [ ]:
import os                                                         # Main functionality features
import time
import timeit
import random
import pandas as pd
import numpy as np
from threading import Timer, Lock
from matplotlib import pyplot as plt                              # Needed for displaying histogram
from matplotlib.ticker import PercentFormatter, MaxNLocator
from scipy.stats import norm
from scipy.optimize import curve_fit
from IPython.display import display, clear_output                 # User interface
from ipywidgets import HTML, Output, VBox, Layout, Label, Image

from rp_overlay import overlay                                    # Red Pitaya libs
import rp

fpga = overlay()
rp.rp_Init()

################################
###    Frontend Functions    ###
################################

def create_message(text, background_color="#ffffff", text_color="#333333"):
    return HTML(value=f'''
        <div style="
            display: flex;
            justify-content: center;
            align-items: center;
            height: auto; /* Adjusted to auto to fit content */
            min-height: 240px; /* Ensures it's not smaller than this but can grow */
            font-size: 34px;
            text-align: center;
            font-weight: bold;
            line-height: 1.5; /* Adjusted line-height for better spacing */
            width: auto;
            background-color: {background_color};
            color: {text_color};
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            margin: 10px 0;
            padding: 20px;
            font-family: 'Arial', sans-serif;
            ">
            <p>{text}</p>
        </div>
    ''', layout=Layout(margin='0 auto', width='50%'))

#d02321
#D52F33
red_message = create_message("Wait for the Green Circle", background_color="#d02321", text_color="#ffffff")
blue_message = create_message("Trigger Happy?<br />Wait for the Green Circle!<br />Press Button to Retry", background_color="#87ceeb")
start_message = create_message("Press the Red Button to Start", background_color="#d02321", text_color="#ffffff")
press_message = create_message("PRESS THE BUTTON", background_color="#19a337", text_color="#ffffff")
gj_message = create_message("Great Job!", background_color="#19a337", text_color="#ffffff")
try_message = create_message("To try again press the Button<br>or<br>Wait 10 Seconds to Display the Leaderboard", background_color="#d02321", text_color="#ffffff")
dnf_message = create_message("Did not finish :(<br>Better luck next time!", background_color="#87ceeb")

def create_two_color_message(text, background_color="#ffffff", text_color_other="#333333", text_color_mid="#000000"):
    return HTML(value=f'''
        <div style="
            display: flex;
            justify-content: center;
            align-items: center;
            height: auto; /* Adjusted to auto to fit content */
            min-height: 240px; /* Ensures it's not smaller than this but can grow */
            font-size: 34px;
            text-align: center;
            font-weight: bold;
            line-height: 1.5; /* Adjusted line-height for better spacing */
            width: auto;
            background-color: {background_color};
            color: {text_color_other};
            border-radius: 10px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            margin: 10px 0;
            padding: 20px;
            font-family: 'Arial', sans-serif;
            ">
                <p>Press the Red Button<br>Measure your reaction time to win<br><span style="color:{text_color_mid};">{text}</span></p>
            </div>
    ''', layout=Layout(margin='0 auto', width='50%'))

hoodie_message = create_two_color_message("a FREE Red Pitaya Hoodie", background_color="#d02321", text_color_other="#ffffff", text_color_mid="#000000")

#Define Circles
def create_colored_circle(color):
    html_circle = HTML(value=f"""
        <div style='
            width:400px; 
            height:400px; 
            background-color:{color}; 
            border-radius:50%; 
            margin:20px auto; 
            box-shadow: 0 4px 8px rgba(0,0,0,0.2);
            display: flex;
            justify-content: center;
            align-items: center;
            font-size: 24px;
            color: #ffffff;
            font-weight: bold;
            '></div>
    """, layout=Layout(margin='0 auto', width='auto'))
    return html_circle


red_circle = create_colored_circle("#d02321")   # 'red'
blue_circle = create_colored_circle("#87ceeb")      # 'blue'
green_circle = create_colored_circle("#19a337")   # 'green'

#Display time and leaderboard
def display_fancy_time(time_ms):
    # Define your HTML content with inline CSS for styling
    html_content = f'''
    <div style="
        display: flex;
        justify-content: center;
        align-items: center;
        height: auto;
        min-height: 100px;
        width: auto
        margin: 10px 0;
        padding: 20px;
        border-radius: 10px;
        background-color: #f0f0f0; /* Light grey background */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* Soft shadow */
        text-align: center;
        font-size: 32px; /* Larger font size */
        font-family: 'Arial', sans-serif;
        color: #333; /* Dark grey text */
        line-height: 1.5;
    ">
        <p>Your reaction time is:<br><strong>{time_ms:.6f} ms</strong></p>
    </div>
    '''
    # Create an HTML widget with the content
    fancy_time_widget = HTML(value=html_content, layout=Layout(margin='0 auto', width='50%'))
    
    display(fancy_time_widget)


# Display time and placement
def display_best_placement_time(time_ms, placement, top_text, background_color="#f0f0f0", text_color="#333"):
    # Define your HTML content with inline CSS for styling
    html_content = f'''
    <div style="
        display: flex;
        justify-content: center;
        align-items: center;
        height: auto;
        min-height: 100px;
        width: auto
        margin: 10px 0;
        padding: 20px;
        border-radius: 10px;
        background-color: {background_color}; /* Light grey background */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* Soft shadow */
        text-align: center;
        font-size: 32px; /* Larger font size */
        font-family: 'Arial', sans-serif;
        color: {text_color}; /* Dark grey text */
        line-height: 1.5;
    ">
        <p>{top_text}You are <strong>{placement}</strong>!<br>Reaction time: <strong>{time_ms:.6f} ms</strong></p>
    </div>
    '''
    # Create an HTML widget with the content
    best_time_widget = HTML(value=html_content, layout=Layout(margin='0 auto', width='50%'))
    
    return best_time_widget




#Display time and leaderboard
def display_contact_info(text, strong_text):
    # Define your HTML content with inline CSS for styling
    html_content = f'''
    <div style="
        justify-content: center;
        align-items: center;
        text-align: center;
        width: auto;
        height: auto;
        margin: 10px 0;
        padding: 10px;
        border-radius: 10px;
        background-color: #f0f0f0; /* Light grey background */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* Soft shadow */
        font-size: 30px; /* Larger font size */
        font-family: 'Arial', sans-serif;
        color: #333; /* Dark grey text */
        line-height: 1.5;
        white-space: nowrap; /* Prevents text wrapping */
        overflow: hidden; /* Hide overflowed text */
        text-overflow: ellipsis; /* Show ellipsis (...) when text overflows */
    ">
        <p>{text}<strong>{strong_text}</strong></p>
    </div>
    '''
    # Create an HTML widget with the content
    contact_info_widget = HTML(value=html_content, layout=Layout(margin='0 auto', width='50%'))
    return contact_info_widget
# height: 50px;


input_message = display_contact_info("Please Enter your information", "")
confirm_message = display_contact_info("Confirm with Enter", "")
contact_info_message = display_contact_info("", "Your Contact Info Will NOT Be Displayed")



def logged_message(name):
    # Define your HTML content with inline CSS for styling
    html_content = f'''
    <div style="
        height: 250px; /* Set the height to 250px */
        display: flex;
        justify-content: center;
        align-items: center;
        margin: 10px 0;
        padding: 20px;
        border-radius: 10px;
        background-color: #d02321; /* Custom background color - d02321 - Red Pitaya logo Red */
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* Soft shadow */
        text-align: center;
        font-size: 34px; /* Larger font size */
        font-family: 'Arial', sans-serif;
        font-weight: bold;
        line-height: 1.5; /* Adjusted line-height for better spacing */
        color: #ffffff; /* White text */
        width: 600px; /* Maintain width at 50% */
        margin-left: auto; /* Center align the div */
        margin-right: auto; /* Center align the div */
    ">
        <p>Hi {name}!<br>Press the Red Button to start!</p>
    </div>
    '''
    # Create an HTML widget with the content
    logged_in_message = HTML(value=html_content, layout=Layout(margin='0 auto'))
    display(logged_in_message)

def display_leaderboard():
    """
    Sort data and display it in a top 10 leaderboard
    """
    with leaderboard_output:
        clear_output(wait=True)
        # Sort the DataFrame and reset the index to get a continuous sequence, excluding the "email" column

        if not ask_email:
            sorted_data = user_data.sort_values(by="Time [ms]", ascending=True, na_position= "last").head(10).reset_index(drop=True)
        else:
            sorted_data = user_data.sort_values(by="Time [ms]", ascending=True, na_position= "last").head(10).reset_index(drop=True).drop(columns=["Email"])

        
        # Add a 'Rank' column that goes from 1 to the length of the sorted_data
        sorted_data['Rank'] = range(1, len(sorted_data) + 1)
        
        # Move 'Rank' column to the first column if it's not already
        col_order = ['Rank'] + [col for col in sorted_data.columns if col != 'Rank']
        sorted_data = sorted_data[col_order]
        
     # Style the DataFrame
        styled_data = sorted_data.style.set_properties(**{
            'text-align': 'center',
        }).set_table_styles([
            {'selector': 'th', 'props': [('text-align', 'center')]},  # This line ensures headers are centered
        ]).set_table_attributes("style='width:50%; margin-left:auto; margin-right:auto;'").hide_index().background_gradient(cmap='Reds', subset='Time [ms]')
    
        display(styled_data)

leaderboard_output = Output(layout=Layout(margin='10px 0', width='100%', overflow='auto'))


def display_image():
    
    with image_output:
        clear_output(wait=True)
         
        img = open('stats.png', 'rb').read()
        wi = Image(value=img, format='png')
        o = Output()
        o.append_display_data(wi)
        display(o)

image_output = Output(layout=Layout(margin='10px 0', width='50%', overflow='auto'))

# Vbox Layouts
vbox_start = VBox([hoodie_message, leaderboard_output],   # start message - leaderboard
                   layout=Layout(display='flex',
                                 flex_flow='column',
                                 align_items='center',
                                 width='100%'))

vbox_stats = VBox([hoodie_message, image_output],         # start message - stats
                   layout=Layout(display='flex',
                                 flex_flow='column',
                                 align_items='center',
                                 width='100%'))


## Backend Function

In [ ]:
###############################
###    Backend Functions    ###
###############################

class UserInterface:
    state = 0

class Button:
    pin = rp.RP_DIO2_P      # rp.RP_DIO4_P (mikroBUS 1 - INT) or rp.RP_DIO2_P (mirkoBUS 2 - INT)
    old_state = 1

##### Timer setup ###### 

class Timekeeper(object):   # Repeatable thread timer
    """
    A reusable thread safe timer implementation
    Source: https://stackoverflow.com/questions/22433394/calling-thread-timer-more-than-once
    """
    start_time = 0.0        # in miliseconds
    end_time = 10000.0      # in miliseconds
    i = 0                   # Iteration
    times = np.zeros(5)     # Default timer values (reaction times)

    def __init__(self, interval_sec, function, *args, **kwargs):
        """
        Create a timer object which can be restarted

        :param interval_sec: The timer interval in seconds
        :param function: The user function timer should call once elapsed
        :param args: The user function arguments array (optional)
        :param kwargs: The user function named arguments (optional)
        """
        self._interval_sec = interval_sec
        self._function = function
        self._args = args
        self._kwargs = kwargs
        # Locking is needed since the '_timer' object might be replaced in a different thread
        self._timer_lock = Lock()
        self._timer = None

    def setup(self, interval_sec, function, *args, **kwargs):
        self._interval_sec = interval_sec
        self._function = function
        self._args = args
        self._kwargs = kwargs

    def start(self, restart_if_alive=True):
        """
        Starts the timer and returns this object [e.g. my_timer = TimerEx(10, my_func).start()]
    
        :param restart_if_alive: 'True' to start a new timer if current one is still alive
        :return: This timer object (i.e. self)
        """
        with self._timer_lock:
            # Current timer still running
            if self._timer is not None:
                if not restart_if_alive:
                    # Keep the current timer
                    return self
                # Cancel the current timer
                self._timer.cancel()
            # Create new timer
            self._timer = Timer(self._interval_sec, self.__internal_call)
            self._timer.start()
        # Return this object to allow single line timer start
        return self

    def cancel(self):
        """
        Cancels the current timer if alive
        """
        with self._timer_lock:
            if self._timer is not None:
                self._timer.cancel()
                self._timer = None

    def is_alive(self):
        """
        :return: True if current timer is alive (i.e not elapsed yet)
        """
        with self._timer_lock:
            if self._timer is not None:
                return self._timer.is_alive()
        return False

    def __internal_call(self):
        # Release timer object
        with self._timer_lock:
            self._timer = None
        # Call the user defined function
        self._function(*self._args, **self._kwargs)


##### Timer functions ###### 

# Changing state if the button is not pressed after timeout
def watchdog_timer(time_watchdog, n):
    """
    Calls watchdog_timer_handler after n seconds.

    Args:
        time_watchdog (TimeKeeper class)  : Holds static variables for reaction times
        n (int) : Number of seconds to call fixed_timer_handler function after
    """

    time_watchdog.setup(n, watchdog_timer_handler)   # Configure fixed timer
    time_watchdog.start(False)

def watchdog_timer_handler():
    """
    Called when the watchdog timer finishes.
    Changes GUI state and stops the repeated measurements.

    """
    global first_time
    global GUI

    GUI.state = 6       # Change to Calculate statistics
    first_time = True   # Preapare the first time run of the next state 

    return 0        # The value cannot be recalled

# Random timer for Red Button
def random_timer(timeKeep):
    """
    Calls rand_timer_handler function after 2-10 seconds.

    Args:
        timeKeep (TimeKeeper class)  : Holds static variables for reaction times
        gui      (UserInteface class): Holds static variable for GUI
    """
    rand_time = round(2 + random.random()*8, 3)                 # Create random time
    timeKeep.setup(rand_time, rand_timer_handler)   # Configure timer
    timeKeep.start(False)

def rand_timer_handler():
    """
    Called when the random timer finishes.
    Changes GUI state to Green Circle state.

    Args:
        gui      (UserInteface class): Holds static variable for GUI
    """
    global first_time
    global GUI
    GUI.state = 3           # Change to Measure Reaction state (Green Circle)
    first_time=True         # Preapare the first time run of the next state

    # The reaction time measurement starts immeadiately after displaying green circle

    return 0        # The value cannot be recalled

# Switching between statistics and leader board
def stats_leader_timer(time_switch_stats_leader, n):
    """
    Calls stastistics_leaderboard_timer_handler after n seconds.

    Args:
        time_switch_stats_leader (TimeKeeper class)  : Holds static variables for reaction times
        n (int) : Number of seconds to call stastistics_leaderboard_timer_handler function after
    """

    time_switch_stats_leader.setup(n, stats_leader_timer_handler)   # Configure fixed timer
    time_switch_stats_leader.start(False)


def stats_leader_timer_handler():
    """
    Called when the stastistics_leaderboard timer finishes.
    Changes GUI state and stops the repeated measurements.

    """
    global first_time
    global leaderboard_nStatistics

    first_time = True                                       # Preapare the first time run to change the displayed data
    leaderboard_nStatistics = not leaderboard_nStatistics   # Negate the flag

    return 0        # The value cannot be recalled


##### Data handling ###### 

def checkUserData(filename, ask_company, ask_email):
    """Checks whether the user data csv file already exists and opens it,
    Otherwise it creates a new file with some new data.

    Returns:
        user_data: pandas.DataFrame containing the collected user data
    """
    # Check if a CSV file already exists
    if not os.path.isfile(filename):
        # user data exists
        user_data = pd.DataFrame({
            "Name"      : ['Miha', 'Crt', 'Lora', 'Nicu', 'Nina'],
            "Time [ms]" : [600.0, 580.0, 590.0, 605.0, 610.0]
        })
        if ask_company:
            user_data.insert(1, "Company", ['Red Pitaya','Red Pitaya','Red Pitaya','Red Pitaya','Red Pitaya'], True)
        if ask_email:
            user_data.insert(1, "Email", ['miha.gjura@redpitaya.com', 'crt.valentincic@redpitaya.com', 'lora.stefanova@redpitaya.com', 'nicu.irimia@redpitaya.com', 'nina.bizjak@redpitaya.com'], True)

        user_data.to_csv(filename, sep=',', index=False, encoding='utf-8')

    # Load data from file
    user_data = pd.read_csv(filename, header= 0)
    
    return user_data


def button_press(state):
    """Tracks whether a button was pressed or not. #! NO DEBOUNCE

    Args:
        state (class Button): Button class containing pin number and old value of the pin from the last reading
    Returns:
        pressed (bool): True when the button was pressed, False when it was not
    """
    new_state = 0
    
    new_state = rp.rp_DpinGetState(state.pin)[1]    # Get new state
    #print(new_state, state.old_state)
    if (state.old_state == 0) and (new_state == 1): # Check if button is pressed
        pressed = True
    else:
        pressed = False

    state.old_state = new_state                     # New state is old state
    #print(new_state, state.old_state)
    
    return pressed


def sort_time_data_save_csv(user_data, destination):
    """
    Sorts the user data by time value and saves it into the CSV file.
    Returns sorted data by time values
    """
    sorted_data = user_data.sort_values(by="Time [ms]", ascending=True, na_position= "last").reset_index(drop=True)
    sorted_data.to_csv(destination, index=False)

    return sorted_data


def get_time_array(user_data, max_x_axis_data):
    """
    Returns the time values as a numpy array from the supplied data frame 
    """
    return user_data[user_data["Time [ms]"] <= max_x_axis_data]["Time [ms]"].to_numpy()


def gauss(x, amp, mu, sigma):
    """
    Gaussian function - data, amplitude, mean value, standard deviation
    """
    return amp * np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))

##### Plot Statistics ###### 
def prepare_data_plot(data, latest_entry_time, max_x_axis_data, x_axis_step):
    """
    Calculates all necessary data to plot the histogram and Gaussian curve fit.
    Needs the latest user data and the latest data index for time
    """

    if (latest_entry_time <= max_x_axis_data):

        # Plot histogram
        fig=plt.figure(figsize=(9,5), dpi= 100, facecolor='w', edgecolor='k', layout="constrained")
        y_hist, bins, patches = plt.hist(data, num_bins, range = [0, max_x_axis_data],color = "#d02321", edgecolor='k',alpha = 0.7) # For percentage add - weights=np.ones(len(data))/len(data)
    
        # Find the patches index where the latest entry time is located
        latest_entry_patches_index = int(latest_entry_time // (max_x_axis_data // num_bins))
        patches[latest_entry_patches_index].set_fc("#000000")   # Change face colour of a particular column
        
        # For percentage - plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))     # Exclude for percentage
        plt.xticks(np.arange(0, max_x_axis_data, step=x_axis_step), rotation=0)
        
        # Plot the optimal curve only if there is a maximum in data
        if (np.nanmax(y_hist) > 1) and (len(data) > 20):
            # Calculate the mean and standard deviation for current data:
            mean, std = norm.fit(data)
            xmin, xmax = plt.xlim()
            
            # Get histogram x vector
            x_hist = np.zeros((num_bins), dtype=float)
                
            for ii in range(num_bins):
                x_hist[ii]=(bins[ii+1]+bins[ii])/2
            
            # Find optimal parameters
            try:
                popt, pcov = curve_fit(gauss, x_hist, y_hist, p0=(np.max(y_hist), mean, std), maxfev=5000)
            except RuntimeError as e:
                pass
            else:
                x = np.linspace(xmin, xmax, num_bins)
                plt.plot(x, gauss(x, *popt), 'k--', linewidth=2, label='Gaussian fit')
            
        plt.xlabel("Reaction time [ms]", fontsize=16)
        plt.ylabel("Users", fontsize=16)
        plt.suptitle(f"Reaction time statistics (N = {int(np.sum(y_hist))})", fontweight="bold", fontsize=20,)
        plt.plot()
        fig.savefig("stats.png", bbox_inches='tight')

# Placement

def find_placement(best_reaction_time, user_data_table):
    """
    Finds the user's placement depending on their reaction time.
    Returns placement string and top10 bool (True if the user is top 10)
    """
    if best_reaction_time <= 10000:
        failed = False
        # Find the user placement (must add one, due to how indexing works)
        placement = user_data_table[user_data_table["Time [ms]"] == best_reaction_time].index[0] + 1

        if placement <= 10:
            top10 = True
            if (placement == 1):
                top10_colour = "#FFD700"
                top10_text_colour = "#000000"
            elif (placement == 2):
                top10_colour = "#C0C0C0"
                top10_text_colour = "#000000"
            elif (placement == 3):
                top10_colour = "#CD7F32"
                top10_text_colour = "#000000"
            else:
                top10_colour = "#9b111e"
                top10_text_colour = "#ffffff"
        else:
            top10 = False
            top10_colour = "#f0f0f0"
            top10_text_colour = "#333333"

        

        if (placement % 10 == 1):
            placement_string = f"{placement}st"
        elif (placement % 10 == 2):
            placement_string = f"{placement}nd"
        elif (placement % 10 == 3):
            placement_string = f"{placement}rd"
        else:
            placement_string = f"{placement}th"
            
        if (10 < placement < 14):
            placement_string = f"{placement}th"
    else:
        placement_string = "DNF"
        top10 = False
        failed = True
        top10_colour = "#f0f0f0"
        top10_text_colour = "#333333"

    return placement_string, top10, top10_colour, top10_text_colour, failed



#############################
####      Variables      ####
#############################

# GUI state keeper
GUI = UserInterface()   # 0 == Result display (Default, Statistics)
                        # 1 == Red circle (Wait)
                        # 2 == Green circle (Press button)
                        # 3 == Info prompt (Name, Company, Email, Time)
                        # 4 == Blue circle (Trigger happy)
                        # 5 == Another one (Repeat measurements)
                        # 6 == Display result (Save data into table) Your Time + Position 

# Keeps the While loop running
program_running = True

# Get the current working directory
cwd = os.getcwd()


# Keeps button states and pin number
red_button = Button()
rp.rp_DpinSetDirection(red_button.pin, rp.RP_IN)


# Ask for contact info
ask_company = False
ask_email = True

name = ""                           # Customer data
company = ""
email = ""
latest_time_index = 0

# User data file name and location
user_data_file = "reaction_time_user_data.csv"
user_data = checkUserData(user_data_file, ask_company, ask_email)

# State change variables
first_time = True                   # First time in a state
GUI.state = 0                       # GUI state
leaderboard_nStatistics = True      # GUI = 1, change between leaderboard and statistics
switch_time = 20                    # Switch time for GUI 1

# Number of button presses per user
num_iter = 1

# Result values and limitations
average_time = 0.0                  # Average time over multiple measurements (max five iterations)
max_timer_value = 100000            # Max timer = 100 seconds (have to drop the max timer values from the array)
timeout = 10                        # Timeout during Reaction Time measurement - max time the green button is displayed
repeat_time = 10


# Graph
num_bins = 100
x_axis_step = 50
max_x_axis_data = 1000

# Prepare graph picutre

time_data = get_time_array(user_data, max_x_axis_data)
prepare_data_plot(time_data, 0, max_x_axis_data, x_axis_step)


# Keeps the times - repeat_time is replaced later
timekeeper = Timekeeper(repeat_time, rand_timer_handler)                               # Change GUI to Green circle
watchdog = Timekeeper(repeat_time, watchdog_timer_handler)                             # GUI = 3, 5 - Auto change to statistics if the button is not pressed
time_stats_leader = Timekeeper(repeat_time, stats_leader_timer_handler)                # GUI = 1 - Switching between leaderboard and statistics

# Logged in
logged_in = False


print("OK")

## Main program

In [ ]:
# Main program
while(program_running):
    #! Waiting State - Display the leaderboard and wait for the button press to start
    if(GUI.state == 0):
        # Check if this is the first time the state is executed
        if first_time:
            first_time = False                  # Clear first-time flag

            if leaderboard_nStatistics:
                display_leaderboard()           # Refresh leaderboard
                clear_output()
                display(vbox_start)  
            else:
                display_image()                 # Refresh image
                clear_output()
                display(vbox_stats)

            stats_leader_timer(time_stats_leader, switch_time)

        # Check if the button is pressed
        if (button_press(red_button) == True):
            time_stats_leader.cancel()          # Cancel switch timer
            GUI.state = 1                       # Go to Data Entry state
            first_time = True                   # Preapare the first time run of the next state

    #! User Data Entry - Waiting for user data entry after button press
    elif(GUI.state == 1):

        # Check if this is the first time the state is executed
        if first_time:
            first_time = False                              # Clear first-time flag
            logged_in = True
            clear_output()
            display(input_message)
            display(contact_info_message)
            display(confirm_message)

            # Input customer info
            name= input("Name and Surname: ")
            if ask_company:
                company = input("Company: ")
            if ask_email:
                email= input("Email: ")
    
            # Prepare data entry
            if ask_company and ask_email:
                newest_entry = {"Name" : name, "Company" : company, "Email" : email, "Time [ms]": max_timer_value}
            elif ask_company:
                newest_entry = {"Name" : name, "Company" : company, "Time [ms]": max_timer_value}
            elif ask_email:
                newest_entry = {"Name" : name, "Email" : email, "Time [ms]": max_timer_value}
            else:
                newest_entry = {"Name" : name, "Time [ms]": max_timer_value}

            clear_output()
            display_leaderboard()
            logged_message(name)
            display(leaderboard_output)

        # Check for the button press
        if (button_press(red_button) == True):
            # User finished data entry and started the timer
            GUI.state = 2                   # Go to Start Timer state
            first_time = True               # Preapare the first time run of the next state

    #! Start Timer - Random wait before starting the measurement
    elif(GUI.state == 2):

        # Check if this is the first time the state is executed
        if first_time:
            first_time = False                  # Clear first-time flag
            time.sleep(0.2)
            clear_output()
            display(red_message, red_circle)    # Display Red Circle
            random_timer(timekeeper)            # Sets up random wait timer

        # If the button is pressed before the random timer ends, go to the 'Trigger Happy' state
        if (button_press(red_button) == True):
            timekeeper.cancel()                 # Cancel the random timer
            GUI.state = 4                       # Go to 'Trigger Happy' state
            first_time = True                   # Preapare the first time run of the next state
            time.sleep(0.2)                     # Debounce delay

    #! Measure Reaction - The user presses button to stop the timer
    elif(GUI.state == 3):

        # Check if this is the first time the state is executed
        if first_time:
            first_time = False                              # Clear first-time flag
            clear_output()
            watchdog_timer(watchdog, timeout)               # Start Watchdog (timeout)
            display(press_message, green_circle)            # Display Green Circle
            timekeeper.start_time = timeit.default_timer()  # Start measuring reaction time
 
        if (button_press(red_button) == True):
            timekeeper.end_time = timeit.default_timer()
            watchdog.cancel()                               # Cancel Watchdog

            GUI.state = 5                                   # Transition to Display Result state
            first_time = True

    #! Trigger Happy - The user pressed the button too early
    elif(GUI.state == 4):

        # Check if this is the first time the state is executed
        if first_time:
            first_time = False                  # Clear first-time flag
            clear_output()
            display(blue_message, blue_circle)  # Display Blue circle
            watchdog.cancel()
            # Inform the user they pressed too early and need to try again

        # User acknowledges and tries again
        if (button_press(red_button) == True):
            GUI.state = 2                       # Go back to the Start Timer state
            first_time = True                   # Preapare the first time run of the next state
            time.sleep(0.2)                     # Debounce delay
            timekeeper.end_time = timeit.default_timer()
            

    #! Another one - Repeat reaction time measurements
    elif (GUI.state == 5):

        # Check if this is the first time the state is executed
        if first_time:                                              
            first_time = False                                                                               # Clear first-time run flag
            timekeeper.times[timekeeper.i] = float((timekeeper.end_time - timekeeper.start_time) * 1000)     # Save latest time into an array
            watchdog_timer(watchdog, repeat_time)                                                            # Set the watchdog timer for repeat_time variable

            clear_output()
            display(gj_message)
            display_fancy_time(timekeeper.times[timekeeper.i])  # Display latest time
            
            # Do not display GUI if there is just one iteration
            if num_iter > 1:
                display(try_message)
                time.sleep(0.5)                                     #Deboounce
            
            # If we are in the final iteration (num_iter time in a row for the same user)
            if timekeeper.i >= (num_iter -1):
                watchdog.cancel()           # Cancel the watchdog
                time.sleep(3)               # Wait for a bit before changing the state
                clear_output()
                GUI.state = 6               # Go to Display Results state
                first_time = True           # Preapare the first time run of the next state
            
            timekeeper.i = timekeeper.i + 1     # Increase iteartion count
        
        # Check for a button press
        if (button_press(red_button) == True):
            watchdog.cancel()
            first_time = True       # Preapare the first time run of the next state
            GUI.state = 2           # Go back to the Start Timer state
            time.sleep(0.2)         # Debounce delay


    #! Display Result - Show reaction time and update leaderboard
    elif(GUI.state == 6):
        if first_time:
            first_time = False
    
            # Find the best (minimum) time out of the attempts
            # Filter out zeros before taking the min to ignore uninitialized values
            valid_times = timekeeper.times[timekeeper.times > 0.0]
            if valid_times.size > 0:
                best_time = np.nanmin(valid_times)
            else:
                best_time = max_timer_value    # Use a default high value if no valid attempts were made
    
            newest_entry["Time [ms]"] = best_time  # Update the best time in the newest entry

            # Update the DataFrame with the new entry or update an existing one
            if ask_company and ask_email:
                index = user_data[(user_data["Name"] == name) & (user_data["Company"] == company) & (user_data["Email"] == email)].index
            elif ask_company:
                index = user_data[(user_data["Name"] == name) & (user_data["Company"] == company)].index
            elif ask_email:
                index = user_data[(user_data["Name"] == name) & (user_data["Email"] == email)].index
            else:
                index = user_data[(user_data["Name"] == name)].index

            if not index.empty:
                # Existing user, update the best time if it's better than the previously stored time
                if user_data.loc[index[0], "Time [ms]"] > best_time:
                    user_data.at[index[0], "Time [ms]"] = best_time
            else:
                # New user, append their best time
                user_data = user_data.append(newest_entry, ignore_index=True)

            # Sort the data and save the updated DataFrame to CSV
            user_data = sort_time_data_save_csv(user_data, user_data_file)

            # Find the user placement
            placement_string, top_ten, top_ten_colour, top_ten_text_colour, failed = find_placement(best_time, user_data)

            #!!!! Use the displaying the leaderboard as a coverup for the long wait for the statistics graph !!!!#
            # Sort the DataFrame and display the leaderboard again
            display_leaderboard()
            # Optionally display a message with the best time
            clear_output()
            
            if failed:
                # Display Did Not Finish message
                vbox_time = VBox([dnf_message, leaderboard_output],
                   layout=Layout(display='flex',
                                 flex_flow='column',
                                 align_items='center',
                                 width='100%'))
                display(vbox_time)
                time.sleep(2)
            elif top_ten:
                # Display best time
                vbox_time = VBox([display_best_placement_time(best_time, placement_string, "Congratulations!<br>", top_ten_colour, top_ten_text_colour), leaderboard_output],
                   layout=Layout(display='flex',
                                 flex_flow='column',
                                 align_items='center',
                                 width='100%'))
                display(vbox_time)
            else:
                # Display placement
                vbox_time = VBox([display_best_placement_time(best_time, placement_string, "", top_ten_colour, top_ten_text_colour), leaderboard_output],
                   layout=Layout(display='flex',
                                 flex_flow='column',
                                 align_items='center',
                                 width='100%'))
                display(vbox_time)

            # Recalculate the statistics
            time_data = get_time_array(user_data, max_x_axis_data)
            prepare_data_plot(time_data, best_time, max_x_axis_data, x_axis_step)   # Latest_entry_time == best_time
            time.sleep(4)
        
            # Reset for the next user or attempt
            GUI.state = 0
            first_time = True
            leaderboard_nStatistics = False     # Show statistics first
            
            timekeeper.times[:] = 0  # Clear recorded times
            timekeeper.i = 0  # Reset attempt counter

In [ ]:
rp.release()